## Wstępna analiza

Wczytujemy plik csv używając biblioteki pandas jako DataFrame, upewniamy się że widoczne będą wszystkie kolumny z danymi, analizujemy wielkość i naturę zbioru danych.

In [2]:
import pandas as pd
df = pd.read_csv('data/final_train.csv')
pd.set_option('display.max_columns', None)

In [3]:
df.shape

(11086, 573)

In [4]:
df.sample(6)

,Unnamed: 0,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16","fBodyAcc-bandsEnergy()-1,16.1","fBodyAcc-bandsEnergy()-1,16.2","fBodyAcc-bandsEnergy()-1,24","fBodyAcc-bandsEnergy()-1,24.1","fBodyAcc-bandsEnergy()-1,24.2","fBodyAcc-bandsEnergy()-1,8","fBodyAcc-bandsEnergy()-1,8.1","fBodyAcc-bandsEnergy()-1,8.2","fBodyAcc-bandsEnergy()-17,24","fBodyAcc-bandsEnergy()-17,24.1","fBodyAcc-bandsEnergy()-17,24.2","fBodyAcc-bandsEnergy()-17,32","fBodyAcc-bandsEnergy()-17,32.1","fBodyAcc-bandsEnergy()-17,32.2","fBodyAcc-bandsEnergy()-25,32","fBodyAcc-bandsEnergy()-25,32.1","fBodyAcc-bandsEnergy()-25,32.2","fBodyAcc-bandsEnergy()-25,48","fBodyAcc-bandsEnergy()-25,48.1","fBodyAcc-bandsEnergy()-25,48.2","fBodyAcc-bandsEnergy()-33,40","fBodyAcc-bandsEnergy()-33,40.1","fBodyAcc-bandsEnergy()-33,40.2","fBodyAcc-bandsEnergy()-33,48","fBodyAcc-bandsEnergy()-33,48.1","fBodyAcc-bandsEnergy()-33,48.2","fBodyAcc-bandsEnergy()-41,48","fBodyAcc-bandsEnergy()-41,48.1","fBodyAcc-bandsEnergy()-41,48.2","fBodyAcc-bandsEnergy()-49,56","fBodyAcc-bandsEnergy()-49,56.1","fBodyAcc-bandsEnergy()-49,56.2","fBodyAcc-bandsEnergy()-49,64","fBodyAcc-bandsEnergy()-49,64.1","fBodyAcc-bandsEnergy()-49,64.2","fBodyAcc-bandsEnergy()-57,64","fBodyAcc-bandsEnergy()-57,64.1","fBodyAcc-bandsEnergy()-57,64.2","fBodyAcc-bandsEnergy()-9,16","fBodyAcc-bandsEnergy()-9,16.1","fBodyAcc-bandsEnergy()-9,16.2",fBodyAcc-energy()-X,fBodyAcc-energy()-Y,fBodyAcc-energy()-Z,fBodyAcc-entropy()-X,fBodyAcc-entropy()-Y,fBodyAcc-entropy()-Z,fBodyAcc-iqr()-X,fBodyAcc-iqr()-Y,fBodyAcc-iqr()-Z,fBodyAcc-kurtosis()-X,fBodyAcc-kurtosis()-Y,fBodyAcc-kurtosis()-Z,fBodyAcc-mad()-X,fBodyAcc-mad()-Y,fBodyAcc-mad()-Z,fBodyAcc-main()-X,fBodyAcc-main()-Y,fBodyAcc-main()-Z,fBodyAcc-max()-X,fBodyAcc-max()-Y,fBodyAcc-max()-Z,fBodyAcc-maxInds-X,fBodyAcc-maxInds-Y,fBodyAcc-maxInds-Z,fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAcc-meanFreq()-X,fBodyAcc-meanFreq()-Y,fBodyAcc-meanFreq()-Z,fBodyAcc-min()-X,fBodyAcc-min()-Y,fBodyAcc-min()-Z,fBodyAcc-skewness()-X,fBodyAcc-skewness()-Y,fBodyAcc-skewness()-Z,fBodyAcc-sma(),fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,"fBodyAccJerk-bandsEnergy()-1,16","fBodyAccJerk-bandsEnergy()-1,16.1","fBodyAccJerk-bandsEnergy()-1,16.2","fBodyAccJerk-bandsEnergy()-1,24","fBodyAccJerk-bandsEnergy()-1,24.1","fBodyAccJerk-bandsEnergy()-1,24.2","fBodyAccJerk-bandsEnergy()-1,8","fBodyAccJerk-bandsEnergy()-1,8.1","fBodyAccJerk-bandsEnergy()-1,8.2","fBodyAccJerk-bandsEnergy()-17,24","fBodyAccJerk-bandsEnergy()-17,24.1","fBodyAccJerk-bandsEnergy()-17,24.2","fBodyAccJerk-bandsEnergy()-17,32","fBodyAccJerk-bandsEnergy()-17,32.1","fBodyAccJerk-bandsEnergy()-17,32.2","fBodyAccJerk-bandsEnergy()-25,32","fBodyAccJerk-bandsEnergy()-25,32.1","fBodyAccJerk-bandsEnergy()-25,32.2","fBodyAccJerk-bandsEnergy()-25,48","fBodyAccJerk-bandsEnergy()-25,48.1","fBodyAccJerk-bandsEnergy()-25,48.2","fBodyAccJerk-bandsEnergy()-33,40","fBodyAccJerk-bandsEnergy()-33,40.1","fBodyAccJerk-bandsEnergy()-33,40.2","fBodyAccJerk-bandsEnergy()-33,48","fBodyAccJerk-bandsEnergy()-33,48.1","fBodyAccJerk-bandsEnergy()-33,48.2","fBodyAccJerk-bandsEnergy()-41,48","fBodyAccJerk-bandsEnergy()-41,48.1","fBodyAccJerk-bandsEnergy()-41,48.2","fBodyAccJerk-bandsEnergy()-49,56","fBodyAccJerk-bandsEnergy()-49,56.1","fBodyAccJerk-bandsEnergy()-49,56.2","fBodyAccJerk-bandsEnergy()-49,64","fBodyAccJerk-bandsEnergy()-49,64.1","fBodyAccJerk-bandsEnergy()-49,64.2","fBodyAccJerk-bandsEnergy()-57,64","fBodyAccJerk-bandsEnergy()-57,64.1","fBodyAccJerk-bandsEnergy()-57,64.2","fBodyAccJerk-bandsEnergy()-9,16","fBodyAccJerk-bandsEnergy()-9,16.1","fBodyAccJerk-bandsEnergy()-9,16.2",fBodyAccJerk-energy()-X,fBodyAccJerk-energy()-Y,fBodyAccJerk-energy()-Z,fBodyAccJerk-entropy()-X,fBodyAccJerk-entropy()-Y,fBodyAccJerk-entropy(

W trakcie analizy dochodzimy do kilku wniosków, mianowicie:

In [5]:
pd.unique(df['Activity'])

array(['STANDING', 'WALKING_UPSTAIRS', 'LAYING', 'WALKING', 'SITTING',
       'WALKING_DOWNSTAIRS'], dtype=object)

1. Mamy 6 aktywności, które nasz model ma rozpoznawać,

In [6]:
df.subject.nunique()

30

2. W badaniu brało udział 30 uczestników,

3. Do trenowania modelu klasyfikacji definitywnie nie potrzebujemy wszystkich cech. Skupiamy się na tych, które zawierają średnią i odchylenie standardowe.

## Oczyszczanie danych

Zanim zaczniemy czyścić zbiór, wyciągamy z niego dane o uczestnikach i aktywnościach, to przyda nam się potem.

In [7]:
subject = df['subject']
activity = df['Activity']

Na początku wybieramy wszystkie kolumny zawierające średnią lub odchylenie standardowe wszystkich z cech.

In [8]:
mean_cols = [col for col in df.columns if 'mean' in col or 'Mean' in col]
std_cols = [col for col in df.columns if 'std' in col or 'Std' in col]

df = pd.concat([df[std_cols], df[mean_cols]], axis = 1)
df.shape

(11086, 87)

In [9]:
573-87, (573-87)/573

(486, 0.8481675392670157)

W ten sposób udało nam się pozbyć 486 cech, co stanowi prawie 85% całego zbioru.

Aktualnie nasz zbiór wygląda tak:

In [10]:
df.sample(4)

,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAcc-meanFreq()-X,fBodyAcc-meanFreq()-Y,fBodyAcc-meanFreq()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccJerk-meanFreq()-X,fBodyAccJerk-meanFreq()-Y,fBodyAccJerk-meanFreq()-Z,fBodyAccMag-mean(),fBodyAccMag-meanFreq(),fBodyBodyAccJerkMag-mean(),fBodyBodyAccJerkMag-meanFreq(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-meanFreq(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,fBodyGyro-meanFreq()-X,fBodyGyro-meanFreq()-Y,fBodyGyro-meanFreq()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
10496,0.016032,-2.836052,3.993116,-5.066653,1.933033,2.093067,-1.513092,-2.890308,-6.050516,4.246825,-0.914943,0.755810,4.426131,6.058817,3.764960,2.950739,5.401974,-2.056362,3.721760,2.917945,0.359691,1.672970,4.336518,1.470095,-3.506829,-0.551104,0.853528,-2.485222,1.621000,3.691703,1.262103,4.041029,0.517117,-1.575067,-0.713835,-2.448516,-1.899877,0.153505,-1.908400,1.785244,-6.715516,-7.392977,-0.525725,6.669373,-3.979844,3.376725,-1.895895,3.501309,-0.096265,-1.938364,-4.636029,7.213393,1.569115,6.490134,3.341126,3.323469,1.656472,-0.536478,0.621643,2.171538,2.981646,2.530705,2.506697,-4.434721,0.520778,1.101252,0.920790,3.978315,-0.845393,2.935254,0.649104,3.473233,1.678105,4.745120,4.108398,-1.180607,3.118851,-1.887043,1.028889,0.626815,1.563005,2.326514,-1.516085,2.241313,2.637281,3.487237,-0.860148
5901,0.782542,-1.315487,4.206194,-4.035635,0.271185,2.355980,-0.838646,-3.015638,-6.016085,4.841528,3.323182,0.806858,4.685420,6.341033,3.881736,3.440171,6.304277,-0.530593,3.919964,2.316887,1.598607,5.369631,5.607410,1.774585,-4.165298,-0.590117,0.852435,-2.666552,5.772904,3.714004,1.424940,4.095446,1.994500,-1.569522,-0.506872,-2.430764,-1.867934,0.063986,-0.198620,4.790472,-11.694040,-7.611051,-1.022491,6.348288,-3.512620,4.101112,-1.913154,4.674751,1.777873,-1.623140,-4.488843,7.092995,1.338015,7.500647,3.978866,3.336202,1.074831,-0.316590,0.481502,1.590608,3.213407,1.445280,2.532178,-5.507217,0.557321,0.581818,0.545036,3.983256,-0.985788,2.863186,0.762282,3.536882,1.781501,5.761345,3.568137,-2.055819,3.154385,-2.277488,1.118089,0.496054,2.331681,2.683877,0.343459,2.434286,2.137163,2.418980,-0.941439
6988,-1.283255,-3.161357,3.870946,-6.250629,2.219342,1.475529,-2.804752,-2.824515,-6.081195,4.650028,1.565835,0.710725,4.134100,5.634211,3.733998,2.540716,4.571304,-2.307205,3.291088,3.347972,-1.509697,3.624113,3.071137,1.183452,-3.191312,-0.471662,0.854424,-2.302259,4.127234,3.679519,1.291738,4.036252,-1.712092,-1.572864,-0.862317,-2.452606,-1.953464,0.323638,1.156601,7.917593,-11.827284,-7.143335,-0.349742,7.110782,-3.944959,3.280561,-1.951914,2.597815,-

Dodatkowo zdecydowaliśmy się pozbyć kolumn, które zawierają słowa kluczowe: 'angle', 'energy' i 'meanFreq'

In [11]:
angle_cols = [col for col in df.columns if 'angle' in col or 'Angle' in col]
energy_cols = [col for col in df.columns if 'energy' in col or 'Energy' in col]
meanFreq_cols = [col for col in df.columns if 'meanFreq' in col]

drop_cols = angle_cols + energy_cols + meanFreq_cols
df = df.drop(drop_cols, axis=1)

In [12]:
df.shape

(11086, 66)

Ostatecznie nasz zbiór ma 66 cech.

Dodajemy kolumny uczestnika i aktywności.

In [13]:
df.insert(0, 'Activity', activity)
df.insert(0, 'Subject', subject)

Pozbywamy się niekompletnych wierszy.

In [14]:
df = df.dropna()

Dane, które użyliśmy do przetestowania modelu

In [15]:
df.head(10)

,Subject,Activity,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccMag-mean(),fBodyBodyAccJerkMag-mean(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroMag-mean(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
0,30,STANDING,0.756296,-1.586674,4.141758,-4.176057,0.585073,2.245282,-1.000675,-2.979941,-6.033122,4.781842,2.565681,0.795967,4.632040,6.325173,3.860434,3.289566,6.187228,-0.830160,3.828060,2.470457,1.326475,4.699153,5.242485,1.709530,-4.047928,-0.567443,0.852718,-2.580826,5.308319,3.741996,1.394575,4.017211,1.669987,-7.592399,-0.928158,6.451225,4.527266,1.396293,-1.777539,7.292838,3.333295,-0.417207,1.684650,1.657815,2.521464,-5.258573,3.992989,-0.873160,2.846826,0.765912,3.545092,1.740686,5.620371,3.652962,-2.029314,3.154375,-2.118152,1.094581,0.533126,2.227442,2.599121,-0.003606,2.246320,2.861260,3.815614,-0.928676
1,9,WALKING_UPSTAIRS,-0.537596,-2.784092,3.677794,-5.817531,1.293797,1.616815,-2.166122,-2.906000,-6.062388,4.653310,1.148811,0.660377,3.794268,5.880477,3.775318,2.227671,5.072585,-1.472494,3.399457,2.920422,-0.530696,3.429869,2.133231,0.925902,-3.601759,-0.491527,0.853675,-2.404219,4.198657,3.720622,1.202847,4.050714,-0.544652,-7.291138,-0.623965,7.141304,3.393216,0.616804,-2.409713,5.965342,3.322065,-0.649755,1.949084,2.082325,2.470419,-3.498200,3.990053,-0.902530,2.853683,1.396476,3.512594,1.801041,4.410433,4.357823,-2.404910,3.088970,-1.823719,1.156491,0.567243,1.192297,2.187126,-1.502168,2.488702,2.391994,5.884982,-0.822617
2,28,LAYING,0.755296,-1.283612,4.217478,-4.063730,0.303414,2.333802,-0.806943,-3.011841,-6.017739,4.845153,3.302522,0.807615,4.705391,6.331799,3.883619,3.456453,6.284308,-0.557388,3.904492,2.332787,1.623716,5.340579,5.605033,1.789814,-4.134095,-0.587106,0.852456,-2.656659,5.780229,3.621391,1.430272,4.146776,2.024442,-7.605039,-1.026274,6.347764,4.652675,1.747315,-1.646547,7.496709,3.335892,-0.329539,1.598720,1.465659,2.531127,-5.514689,3.922214,-0.946887,2.876148,0.782550,3.540643,1.812028,5.735402,3.573648,-2.063205,3.135899,-2.219777,1.116531,0.496151,2.328004,2.666988,0.338996,4.616200,1.336379,0.032001,-0.940610
3,25,WALKING,0.149588,-2.796041,3.922191,-4.776575,1.763192,1.876670,-1.633627,-2.920310,-6.048324,4.737144,1.710101,0.746790,4.400935,6.119588,3.769096,2.769627,5.735377,-1.815135,3.567625,2.780547,0.387614,3.929503,4.154581,1.480092,-3.555616,-0.541816,0.853448,-2.497160,4.913941,3.698913,1.319806,4.098944,0.550414,-7.456240,-0.551743,6.816364,4.004257,0.307571,-2.173669,6.682904,3.326199,-0.521936,1.782375,1.919775,2.504932,-4.642801,4.011555,-0.971376,2.884066,0.826285,3.501259,1.645810,4.927229,4.063273,-1.966041,3.131273,-2.128185,1.109825,0.344011,2.133466,2.319992,-0.632525,2.254465,2.830937,3.575867,-0.866937
4,28,LAYING,0.755296,-1.283612,4.217478,-4.063730,0.303414,2.333802,-0.806943,-3.011841,-6.017739,4.845153,3.302522,0

In [16]:
df.shape

(9826, 68)

Ostatecznie zdecydowaliśmy się zgrupować wszystkich uczestników badania razem z aktywnościami, w których brali udział

In [17]:
grouped = df.groupby(['Subject', 'Activity'])

Wyciągamy średnią z wszystkich cech, dla każdego uczestnika względem każdej z badanych aktywności.

In [18]:
df = grouped.mean().reset_index()

I tak prezentują się nasze zgrupowane oczyszczone dane, które użyliśmy do trenowania modelu:

In [19]:
df.head(10)

,Subject,Activity,fBodyAcc-std()-X,fBodyAcc-std()-Y,fBodyAcc-std()-Z,fBodyAccJerk-std()-X,fBodyAccJerk-std()-Y,fBodyAccJerk-std()-Z,fBodyAccMag-std(),fBodyBodyAccJerkMag-std(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroMag-std(),fBodyGyro-std()-X,fBodyGyro-std()-Y,fBodyGyro-std()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerkMag-std(),tBodyAccMag-std(),tBodyGyro-std()-X,tBodyGyro-std()-Y,tBodyGyro-std()-Z,tBodyGyroJerk-std()-X,tBodyGyroJerk-std()-Y,tBodyGyroJerk-std()-Z,tBodyGyroJerkMag-std(),tBodyGyroMag-std(),tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAccMag-std(),fBodyAcc-mean()-X,fBodyAcc-mean()-Y,fBodyAcc-mean()-Z,fBodyAccJerk-mean()-X,fBodyAccJerk-mean()-Y,fBodyAccJerk-mean()-Z,fBodyAccMag-mean(),fBodyBodyAccJerkMag-mean(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroMag-mean(),fBodyGyro-mean()-X,fBodyGyro-mean()-Y,fBodyGyro-mean()-Z,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerkMag-mean(),tBodyAccMag-mean(),tBodyGyro-mean()-X,tBodyGyro-mean()-Y,tBodyGyro-mean()-Z,tBodyGyroJerk-mean()-X,tBodyGyroJerk-mean()-Y,tBodyGyroJerk-mean()-Z,tBodyGyroJerkMag-mean(),tBodyGyroMag-mean(),tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-mean()
0,1,LAYING,0.772614,-1.312344,4.192530,-4.032061,0.269285,2.358421,-0.846795,-3.015795,-6.016373,4.840281,3.258072,0.812411,4.702018,6.337974,3.882237,3.413915,6.308148,-0.525414,3.923883,2.313815,1.587704,5.311617,5.727599,1.787036,-4.157940,-0.591095,0.852431,-2.665856,5.757636,3.661552,1.382972,4.096850,1.981498,-7.609869,-1.028226,6.352840,4.680438,1.787605,-1.614792,7.501614,3.336300,-0.316643,1.596296,1.462809,2.533710,-5.511727,4.004645,-0.940913,2.878737,0.759630,3.542436,1.782693,5.766453,3.577747,-2.080221,3.161045,-2.288669,1.115865,0.482753,2.361614,2.685096,0.308312,4.625978,0.976564,1.935458,-0.939993
1,1,SITTING,0.798087,-1.307646,4.214334,-4.022398,0.260490,2.365289,-0.804229,-3.016613,-6.015543,4.839362,3.315200,0.812666,4.674744,6.346444,3.882630,3.458881,6.316179,-0.517250,3.928543,2.309412,1.642377,5.360378,5.732179,1.762811,-4.172653,-0.591411,0.852416,-2.669868,5.749989,3.708552,1.383187,4.095652,2.046695,-7.614962,-1.029447,6.334512,4.690515,1.797773,-1.607954,7.523649,3.336402,-0.312172,1.598449,1.450545,2.533741,-5.466379,3.990923,-0.947507,2.877263,0.770075,3.542441,1.787094,5.776273,3.559307,-2.028662,3.155999,-2.094855,1.120201,0.487802,2.265538,2.690316,0.293439,2.469496,2.116727,2.545397,-0.942768
2,1,STANDING,0.802282,-1.294112,4.213188,-4.021302,0.268362,2.364522,-0.805330,-3.016604,-6.015761,4.848332,3.322622,0.812209,4.704193,6.347743,3.883326,3.457400,6.317122,-0.524806,3.928251,2.309848,1.641747,5.367754,5.721503,1.789533,-4.166584,-0.591289,0.852426,-2.668567,5.814117,3.730135,1.393746,4.112111,2.045944,-7.615632,-1.029612,6.333684,4.691742,1.788077,-1.608083,7.524775,3.336386,-0.313828,1.588399,1.447334,2.533555,-5.520074,3.995227,-0.954341,2.878280,0.769440,3.540677,1.786851,5.774295,3.556647,-2.066865,3.140634,-2.193088,1.117594,0.495941,2.305471,2.688205,0.355205,2.297586,3.114082,3.940857,-0.943168
3,1,WALKING,-0.424408,-3.359302,3.828758,-5.753965,2.741275,1.604912,-2.147099,-2.845857,-6.102188,4.649459,1.876334,0.654530,4.139078,5.906645,3.724877,2.532573,5.008291,-2.736016,3.396478,3.189624,-0.573249,3.977516,1.755679,1.202210,-3.151973,-0.423273,0.854326,-2.202167,4.036936,3.664303,1.380629,4.062471,-0.595396,-7.290344,-0.357401,7.003724,3.176021,-0.849263,-2.406089,5.877996,3.316642,-0.889056,2.024332,2.024761,2.452966,-3.798914,3.976428,-0.940502,2.874554,0.825027,3.550227,1.864683,4.181109,4.350953,-2.057952,3.120536,-2.255048,1.119887,0.663040,2.172364,1.854422,-1.172223,2.311666,3.121396,4.318224,-0.823651
4,1,WALKING_DOWNSTAIRS,-0.938943,-3.193383,3.840144,-5.907574,2.474086,1.407951,-2.649033,-2.837071,-6.111458,4.659888,1.507965,0.

In [20]:
df.shape

(180, 68)

## Model

Do implementacji modelu użyliśmy biblioteki TensorFlow 

### Definicja modelu

In [ ]:
# build model: 66 -> 150 -> 80 -> 6
model = tf.kras.models.Sequential([
        keras.layers.Dense(150, input_dim=66, activation="relu"),
        keras.layers.Dense(80, activation="relu"),
        keras.layers.Dense(6, activation="softmax")
])

Jest to prosty model. Używa dwóch warstw ukrytych z funkcjami aktywacji ReLU. 

Na ostatniej warstwie jest użyty softmax. Dzięki niej, wartości otrzymane są jak najbardziej czytelne. Sumują się one w przybliżeniu do 1 oraz wartość największa mówi nam, która z czynności jest prawdopodobnie wykonywana.

Dzięki temu możemy w bardziej skomplikowanych implementacjach użyć na przykład drugiej największej wartości.

### Kompilacja modelu

In [ ]:
model.compile(optimizer="adam", loss="MSE", metrics=['accuracy'])

### Trenowanie

Następnym krokiem było trenowanie. Użyliśmy do tego zgrupowanych oczyszczonych danych (6 aktywności na 30 osób - 180 pomiarów)

In [66]:
model.fit(X, Y, epochs=200)

Epoch 1/200
6/6 [==============================] - 0s 609us/step - loss: 0.1620 - accuracy: 0.2444
Epoch 2/200
6/6 [==============================] - 0s 834us/step - loss: 0.1320 - accuracy: 0.3889
Epoch 3/200
6/6 [==============================] - 0s 500us/step - loss: 0.1183 - accuracy: 0.4167
Epoch 4/200
6/6 [==============================] - 0s 834us/step - loss: 0.1049 - accuracy: 0.6444
Epoch 5/200
6/6 [==============================] - 0s 668us/step - loss: 0.0992 - accuracy: 0.6389
Epoch 6/200
6/6 [==============================] - 0s 832us/step - loss: 0.0966 - accuracy: 0.5833
Epoch 7/200
6/6 [==============================] - 0s 499us/step - loss: 0.0904 - accuracy: 0.6667
Epoch 8/200
6/6 [==============================] - 0s 668us/step - loss: 0.0896 - accuracy: 0.6056
Epoch 9/200
6/6 [==============================] - 0s 667us/step - loss: 0.0990 - accuracy: 0.5944
Epoch 10/200
6/6 [==============================] - 0s 666us/step - loss: 0.0926 - accuracy: 0.6167
Epoch 11/

6/6 [==============================] - 0s 835us/step - loss: 0.0147 - accuracy: 0.9333
Epoch 165/200
6/6 [==============================] - 0s 501us/step - loss: 0.0179 - accuracy: 0.9111
Epoch 166/200
6/6 [==============================] - 0s 832us/step - loss: 0.0155 - accuracy: 0.9333
Epoch 167/200
6/6 [==============================] - 0s 833us/step - loss: 0.0160 - accuracy: 0.9389
Epoch 168/200
6/6 [==============================] - 0s 499us/step - loss: 0.0236 - accuracy: 0.8889
Epoch 169/200
6/6 [==============================] - 0s 667us/step - loss: 0.0193 - accuracy: 0.9167
Epoch 170/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0168 - accuracy: 0.9389
Epoch 171/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0149 - accuracy: 0.9500
Epoch 172/200
6/6 [==============================] - 0s 834us/step - loss: 0.0149 - accuracy: 0.9389
Epoch 173/200
6/6 [==============================] - 0s 501us/step - loss: 0.0131 - accuracy: 0.9500
Epoch 17

### Dokładność

Po przetrenowaniu modelu na wcześniejszych danych, sprawdziliśmy dokładność modelu

In [25]:
model.evaluate(X_2, Y_2, verbose=1)


 model evaluation:
308/308 [==============================] - 0s 404us/step - loss: 0.0198 - accuracy: 0.9232


[0.019759194925427437, 0.9231630563735962]

Osiągnęliśmy dokładność rzędu 92% na oczyszczonych danych (wyeliminowanie wierszy z NaNami (9826 pomiarów))

Gotowy model zapisaliśmy do cp.ckpt